#### [ DL CUSTOM MODEL ]

** 모듈로딩 <hr>

In [2]:
import torch                        ## 텐서 및 기본 함수들 관련 모듈
import torch.nn as nn               ## 인공신경망 관련 층과 알고리즘 클래스 모듈
import torch.nn.functional as F     ## 인공신경망 관련 함수들 모듈

** 모델 설계 <hr>

[1]  환경 상황 예측 모델  
- 피쳐 : 3개 
- 타겟 : 연속형 1개
- 학습 : 지도학습 + 회귀 
- 알고리즘 : 인공신경망 계열 - DNN

In [ ]:
## 모델 클래스 설계 ----------------------------------------------------------------------
## 회귀용 커스텀 모델 
## 클래스이름 :  CustomRegressionModel
## 부모클래스 :  nn.Module
## 모델층구성    입력신호/피쳐수       출력신호수/퍼셉트론수          활성화함수
## - 입력층  :       in_in                in_out                  ReLU
##                   _____________________|
##                  | 
##                  ▼
## - 은닉층  :      in_out                h1_out                  ReLU 
##                   _____________________|
##                  | 
##                  ▼
## - 은닉층  :      h1_out                h2_out                  ReLU
##                   _____________________|
##                  | 
##                  ▼
## - 출력층  :     h2_out                 out_out                   X 
## - -----------------------------------------------------------------------------------

In [ ]:
class CustomRegressionModel(nn.Module):
    ## 모델 층 구성 관련 초기화
    def __init__(self, in_in, in_out, h1_out, h2_out, out_out):
        super().__init__()
        self.in_layer  = nn.Linear(in_in,  in_out)
        self.h_layer1  = nn.Linear(in_out, h1_out)
        self.h_layer2  = nn.Linear(h1_out, h2_out)
        self.out_layer = nn.Linear(h2_out, out_out)

    ## 학습 즉, 순방향 학습 진행 메서드 forward : 필수 오버라이딩 메서드 
    ## 매개변수 data : 학습용 데이터 전달
    def forward(self, data):
        # 입력 => 입력층 학습
        out=self.in_layer(data)  ## 출력 f1*w11+f2*w12+f3*W13+b1, ..., f1*w501+f2*w502+f3*W503+b50
        out=F.relu(out)

        # 은닉층 
        out=self.h_layer1(out)  ## 출력 f1*w11+....+f50*W150+b1, ..., f1*w301+...+f50*W3050+b30
        out=F.relu(out)

        # 은닉층 
        out=self.h_layer2(out)  ## 출력 f1*w11+....+f30*W130+b1, ..., f1*w101+...+f30*W1030+b10
        out=F.relu(out)

        # 출력층
        out=self.out_layer(out)  ## 출력 f1*w11+....+f10*W110+b1

        return out 


In [13]:
## 모델 설계 체크 
model = MyCustomModel()
model(torch.FloatTensor([[22,33,44]]))  # (1,3)

tensor([[0.4584]], grad_fn=<AddmmBackward0>)

In [15]:
## 모델 층별 파라미터 즉, W와 b 
for name, param in model.named_parameters():
    print(f'\n[{name}-{param.shape}]===================')
    print(param)


[in_layer.weight-torch.Size([50, 3])]===================
Parameter containing:
tensor([[-0.0667,  0.0607,  0.2364],
        [ 0.1233, -0.1247, -0.1333],
        [ 0.5054,  0.0020, -0.5261],
        [-0.4800,  0.5768,  0.3839],
        [-0.0262,  0.1622, -0.3054],
        [ 0.0497, -0.3429, -0.0969],
        [-0.4499,  0.4305, -0.5295],
        [ 0.3241, -0.1629, -0.0916],
        [ 0.3487,  0.3106,  0.3828],
        [ 0.0946, -0.3812,  0.2936],
        [ 0.2678,  0.0833, -0.3720],
        [ 0.5546, -0.3717, -0.1074],
        [-0.3104,  0.1647, -0.4484],
        [ 0.4710, -0.5562,  0.5709],
        [-0.5624, -0.5365, -0.0464],
        [-0.3280, -0.3476,  0.3593],
        [-0.4241, -0.2115, -0.4413],
        [ 0.5580, -0.3823, -0.5667],
        [-0.0739, -0.1248,  0.5425],
        [-0.5026,  0.1532, -0.0587],
        [-0.4269,  0.3186, -0.5370],
        [ 0.5219, -0.0402,  0.4023],
        [ 0.5487,  0.2996,  0.1807],
        [ 0.4356,  0.0673, -0.1241],
        [ 0.4870, -0.2060, -0.02

[2]  이미지 분류 모델  
- 이미지 크기 : 30 * 30 , 흑백이미지 
- 피쳐 : 900 (30*30*1)
- 타겟 : 자동차와 나무 분류
- 학습 : 지도학습 - 분류 => 이진분류

In [ ]:
## 모델 클래스 설계 ----------------------------------------------------------------------
## 클래스이름 :  CutomImgModel
## 부모클래스 :  nn.Module
## 모델층구성    입력신호/피쳐수       출력신호수/퍼셉트론수      활성화함수
## - 입력층  :       900                   250              ReLU
##                   _____________________|
##                  | 
##                  ▼
## - 은닉층  :     250                    150               ReLU 
##                   _____________________|
##                  | 
##                  ▼
## - 은닉층  :     150                    70                 ReLU
##                   _____________________|
##                  | 
##                  ▼
## - 출력층  :      70                     1                 simgoid 
## - -----------------------------------------------------------------------------------

In [19]:
## 이미지 분류 모델 클래스 설계 
class CustomImgModel(nn.Module):
    ## 모델 설계 초기화
    def __init__(self):
        super().__init__()
        self.in_layer  = nn.Linear(900, 250)
        self.h_layer1  = nn.Linear(250, 150)
        self.h_layer2  = nn.Linear(150, 100)
        self.h_layer3  = nn.Linear(100, 50)
        self.out_layer = nn.Linear(50,  1)

    ## 학습 즉, 순방향 학습 진행 메서드 forward : 필수 오버라이딩 메서드 
    ## 매개변수 data : 학습용 데이터 전달
    def forward(self, data):
        # 입력 => 입력층 학습
        out=self.in_layer(data)  ## 출력 f1*w11+f2*w12+f3*W13+b1, ..., f1*w501+f2*w502+f3*W503+b50
        out=F.relu(out)

        # 은닉층 
        out=self.h_layer1(out)  ## 출력 f1*w11+....+f50*W150+b1, ..., f1*w301+...+f50*W3050+b30
        out=F.relu(out)

        # 은닉층 
        out=self.h_layer2(out)  ## 출력 f1*w11+....+f30*W130+b1, ..., f1*w101+...+f30*W1030+b10
        out=F.relu(out)

        # 은닉층 
        out=self.h_layer3(out)  ## 출력 f1*w11+....+f30*W130+b1, ..., f1*w101+...+f30*W1030+b10
        out=F.relu(out)

        # 출력층
        out=self.out_layer(out)  ## 출력 f1*w11+....+f10*W110+b1
        out=F.sigmoid(out)       ## 이진분류 AF 

        return out 

In [ ]:
## 이미지 분류 모델 클래스 설계 
class CustomImgModel(nn.Module):
    ## 모델 설계 초기화
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(900, 250),
            nn.ReLU(),
            nn.Linear(250, 150),
            nn.ReLU(),
            nn.Linear(150, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 1),
            nn.Sigmoid()
        )

    ## 학습 즉, 순방향 학습 진행 메서드 forward : 필수 오버라이딩 메서드 
    ## 매개변수 data : 학습용 데이터 전달
    def forward(self, data):
        return self.layers(data) 
    
    